In [3]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import os

# Paths to your dataset
train_dir = 'Pictures/Train'
val_dir = 'Pictures/Val'
test_dir = 'Pictures/Test'

# Parameters
IMG_SIZE = (224, 224)  # Image dimensions
BATCH_SIZE = 32        # Batch size
EPOCHS = 30            # Number of epochs
NUM_CLASSES = len(os.listdir(train_dir))  # Count classes dynamically

# Data Generators with enhanced augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Important for evaluation
)

# Load DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune: Unfreeze top layers of DenseNet121
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Better for DenseNet
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(0.01))(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for Early Stopping and Learning Rate Scheduling
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Stop after 5 epochs with no improvement
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,  # Reduce LR if no improvement in 3 epochs
    min_lr=1e-7
)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    verbose=1,
    callbacks=[early_stopping, lr_scheduler]
)

# Save the model
model.save('P_densenet121_medical_model_3.keras')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predict on test data (optional, for detailed evaluation)
predictions = model.predict(test_generator)
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Print class mapping
class_indices = train_generator.class_indices
class_mapping = {v: k for k, v in class_indices.items()}
print("Class Mapping:", class_mapping)


Found 33522 images belonging to 16 classes.
Found 7505 images belonging to 16 classes.
Found 4634 images belonging to 16 classes.
Epoch 1/30
  14/1048 ━━━━━━━━━━━━━━━━━━━━ 1:27:19 5s/step - accuracy: 0.2603 - loss: 9.2858

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/functional_2_1/pool4_conv_1/convolution/Conv2DBackpropFilter defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a primitive, in file tensorflow/core/kernels/mkl/mkl_conv_grad_filter_ops.cc:685
	 [[{{node StatefulPartitionedCall/gradient_tape/functional_2_1/pool4_conv_1/convolution/Conv2DBackpropFilter}}]] [Op:__inference_one_step_on_iterator_241087]